## Embeddings

Cada modelo tiene todos los vídeos y la etiqueta en la columna **shoot_zone**,  donde lanzamiento a la derecha es 0, al centro es 1 y a la izquierda  es 2. 

| Fichero                      | Modelo          | Comentarios sobre “\_SUFIJO”                              |
| ---------------------------- | --------------- | --------------------------------------------------------- |
| **baseline\_CASIAB.csv**     | Baseline        | Versión “base” (p.ej. GEINet simple) entrenada en CASIA-B |
| **baseline\_OUMVLP.csv**     | Baseline        | Igual que el anterior, pero pre-entrenado en OU-MVLP      |
| **gaitgl.csv**               | GaitGL          | GaitGL estándar (dataset por defecto, p.ej. CASIA-B)      |
| **gaitgl\_OUMVLP.csv**       | GaitGL          | Pre-entrenado en OU-MVLP                                  |
| **gaitgl\_GREW\.csv**        | GaitGL          | Pre-entrenado en GREW                                     |
| **gaitgl\_GREW\_BNNeck.csv** | GaitGL + BNNeck | Mismo que el anterior, con cuello de batch-norm extra     |
| **gaitpart.csv**             | GaitPart        | GaitPart estándar                                         |
| **gaitpart\_OUMVLP.csv**     | GaitPart        | Pre-entrenado en OU-MVLP                                  |
| **gaitpart\_GREW\.csv**      | GaitPart        | Pre-entrenado en GREW                                     |
| **gaitset.csv**              | GaitSet         | GaitSet estándar                                          |
| **gaitset\_OUMVLP.csv**      | GaitSet         | Pre-entrenado en OU-MVLP                                  |
| **gaitset\_GREW\.csv**       | GaitSet         | Pre-entrenado en GREW                                     |
| **gln\_phase1.csv**          | GLN (fase 1)    | Primer bloque/fase de extracción del modelo “GLN”         |
| **gln\_phase2.csv**          | GLN (fase 2)    | Fase de refinamiento o bloque final del mismo “GLN”       |


In [1]:
import os
import pandas as pd

# 1. Directorio con los CSV
data_dir = "Gait_Embeddings_good/"

# 2. Listar sólo los archivos .csv
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
print("Archivos encontrados:", csv_files)

# 3. Leer cada CSV en un DataFrame de pandas
dfs = {}
for fname in csv_files:
    path = os.path.join(data_dir, fname)
    dfs[fname] = pd.read_csv(path)

# 4. Explorar cada DataFrame
for name, df in dfs.items():
    print(f"\n=== {name} ===")
    print("Shape:", df.shape)                  # filas × columnas
    print("Columnas:", df.columns.tolist())    # lista de nombres
    print("Primeras 5 filas:")
    print(df.head().to_string(index=False))    # muestra las primeras filas

    # Opcional: ver tipo de datos y memoria
    print("\nInfo:")
    print(df.info())
    print("\nDescripción estadística de columnas numéricas:")
    print(df.describe().T)  # transpuesta para leer mejor


Archivos encontrados: ['baseline_CASIAB.csv', 'baseline_OUMVLP.csv', 'gaitgl.csv', 'gaitgl_GREW.csv', 'gaitgl_GREW_BNNeck.csv', 'gaitgl_OUMVLP.csv', 'gaitpart.csv', 'gaitpart_GREW.csv', 'gaitpart_OUMVLP.csv', 'gaitset.csv', 'gaitset_GREW.csv', 'gaitset_OUMVLP.csv', 'gln_phase1.csv', 'gln_phase2.csv']


C:\Users\Acer\AppData\Local\Temp\ipykernel_2436\3135675460.py:15: DtypeWarning: Columns (129) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[fname] = pd.read_csv(path)



=== baseline_CASIAB.csv ===
Shape: (13392, 261)
Columnas: ['step', 'feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48', 'feat_49', 'feat_50', 'feat_51', 'feat_52', 'feat_53', 'feat_54', 'feat_55', 'feat_56', 'feat_57', 'feat_58', 'feat_59', 'feat_60', 'feat_61', 'feat_62', 'feat_63', 'feat_64', 'feat_65', 'feat_66', 'feat_67', 'feat_68', 'feat_69', 'feat_70', 'feat_71', 'feat_72', 'feat_73', 'feat_74', 'feat_75', 'feat_76', 'feat_77', 'feat_78', 'feat_79', 'feat_80', 'feat_81', 'feat_82', 'feat_83', 'feat_84', 'feat_8

## Metodología de desarrollo

1️⃣ datos → 2️⃣ preprocesado y normalización → 3️⃣ split → 4️⃣ Dataset/DataLoader (+ collate) → 5️⃣ modelo → 6️⃣ entrenamiento (función de pérdida y optimizador) → 7️⃣ evaluación → 8️⃣ ajuste → 9️⃣ guardado.

- Entender y explorar los datos

- Inspecciona las columnas, tipos de variables, balance de clases, valores faltantes y rangos.

- Visualiza distribuciones y posibles outliers.

- Limpieza y preprocesado

- Trata valores faltantes (imputación o eliminación).

- Normalización / escalado

- Aplica Min–Max o Z-score (standarización) para que todas las características queden en un rango controlado y evites que alguna domine el entrenamiento.

- En embeddings, a veces se usa L₂-norm para cada vector si quieres que tengan norma unidad.

- Dividir en train / validation / test

- Reserva al menos un 10–20 % para test “final”.

- Dentro del train crea validación (p. ej. 80/20 o K-fold) para ajustar hiperparámetros sin tocar el test.

- Definir Dataset y DataLoader para construir batches

- Definir el modelo

- Elegir función de pérdida y optimizador

- Bucle de entrenamiento

- Ajuste de hiperparámetros

- Guardado y almacenado de los pesos con torch.save(model.state_dict(), 'modelo.pt').







### Librerías 

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence

In [2]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)

Number of GPU:  1
GPU Name:  NVIDIA GeForce GTX 1650
Using device: cuda


### Guardado de los Modelos

In [3]:

def save_model(model, path: str):
    """
    Guarda en 'path' únicamente los pesos (state_dict) de `model`.
    """
    torch.save(model.state_dict(), path)
    print(f"Modelo guardado en {path}\n")


def load_model(model_class, path, device=DEVICE, **model_kwargs):
    """
    Carga un modelo de cualquier clase PyTorch definida por el usuario.
    
    Parámetros:
    - model_class: la clase del modelo (MLPClassifier, LSTMClassifier, TransformerClassifier, etc.)
    - path:        ruta al archivo .pth con state_dict()
    - device:      dispositivo donde cargar el modelo (ej. DEVICE)
    - **model_kwargs: argumentos para instanciar la clase de modelo 
                      (input_dim, hidden_dim, num_layers, ...)
    """
    # Instancia la arquitectura con los kwargs
    model = model_class(**model_kwargs).to(device)
    # Carga pesos entrenados
    state_dict = torch.load(path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()
    return model

-----------------------------------------------------------------------
## Modelo MLP

In [4]:
#MLP sencillo con pooling previo
class MLPClassifier(nn.Module):
    def __init__(self, input_dim):
        """
        input_dim: dimensión D del embedding tras pooling
        """
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 3)  # 3 clases: derecha, centro, izquierda
        )

    def forward(self, x):
        # x: (batch_size, D)
        return self.net(x)


## Preparación de datos, split y bucle de entrenamiento: MLP
Al preparar los datos para el MLP, primero agrupamos todas las filas del CSV que pertenecen a un mismo video_ID en una matriz de tamaño (T, D), donde T es el número de frames de ese vídeo y D las 256 características por frame. A continuación aplicamos mean‐pooling o max‐pooling sobre el eje temporal T, colapsando cada matriz a un único vector de dimensión (D,) que resume toda la aproximación del jugador al penalti. Ese conjunto de vectores —uno por vídeo— se divide de forma estratificada en train y test, de modo que en el entrenamiento el DataLoader extrae batches de tamaño fijo (por ejemplo 32) con esos vectores y sus etiquetas, y así el MLP aprende a clasificar la dirección del lanzamiento usando esos resúmenes globales.

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import TensorDataset, DataLoader


def prepare_mlp_data(df, pooling, norm, test_size=0.1, pca_components=50):
    """
    1) Pooling temporal (mean o max) para cada video_ID -> vector (D,)
    2) Split 90/10 estratificado
    3) Normalización (MinMax o L2) *solo* con parámetros del train
    4) PCA (.fit en train, .transform en train y test)
    5) Devolver TensorDatasets para DataLoader
    """
    # 1) Pooling
    feat_cols = [c for c in df.columns if c.startswith('feat_')]
    seqs, labs = [], []
    for vid, grp in df.groupby('video_ID'):
        arr = grp[feat_cols].values.astype(np.float32)  # (T, D)
        vec = arr.mean(axis=0) if pooling == 'mean' else arr.max(axis=0)
        seqs.append(vec)
        labs.append(int(grp['shoot_zone'].iloc[0]))
    X = np.vstack(seqs)  # (N, D)
    y = np.array(labs, dtype=np.int64)

    # 2) Split estratificado 90/10
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)

    # 3) Normalización: FIT en train, TRANSFORM en ambos
    if norm == 'minmax':
        scaler = MinMaxScaler().fit(X_tr)
        X_tr = scaler.transform(X_tr)
        X_te = scaler.transform(X_te)
    elif norm == 'L2':
        # normalize devuelve array numpy
        X_tr = normalize(X_tr, norm='l2')
        X_te = normalize(X_te, norm='l2')

    # 4) PCA dinámico
    n_samples, n_features = X_tr.shape
    k = min(pca_components, n_features, n_samples)  # Asegurar que no exceda el número de características 
    pca = PCA(n_components=k, random_state=42).fit(X_tr)
    X_tr = pca.transform(X_tr)
    X_te = pca.transform(X_te)

    # 5) TensorDatasets
    tr_ds = TensorDataset(torch.from_numpy(X_tr).float(), torch.from_numpy(y_tr))
    te_ds = TensorDataset(torch.from_numpy(X_te).float(), torch.from_numpy(y_te))
    
    return tr_ds, te_ds



def run_training(model, train_loader, test_loader, epochs=20, lr=1e-3):
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4) # REVISAR
    loss_fn   = nn.CrossEntropyLoss()
    history   = {'train_loss':[], 'test_acc':[], 'test_f1':[]}

    for ep in range(1, epochs+1):
        # entrenamiento
        model.train()
        total_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            out = model(xb)
            loss = loss_fn(out, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * yb.size(0)
        history['train_loss'].append(total_loss / len(train_loader.dataset))

        # evaluación
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for xb, yb in test_loader:
                xb = xb.to(DEVICE)
                preds = model(xb).argmax(dim=1).cpu().numpy()
                all_preds.append(preds)
                all_labels.append(yb.numpy())
        all_preds  = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)
        acc = accuracy_score(all_labels, all_preds)
        f1  = f1_score(all_labels, all_preds, average='macro')
        history['test_acc'].append(acc)
        history['test_f1'].append(f1)

        print(f"Ep{ep:02d} | loss {history['train_loss'][-1]:.4f} "
              f"| acc {acc:.4f} | f1_macro {f1:.4f}")

    return history


## Entrenamiento

In [ ]:
import os
import pandas as pd
from torch.utils.data import DataLoader

DATA_DIR = "Gait_Embeddings_good"
POOLS    = ['mean', 'max']
NORMS    = ['minmax', 'L2']
EPOCHS   = 500
BATCH    = 32
LOAD_MODEL = False  # Si True, carga un modelo preentrenado en lugar de entrenar uno nuevo

PATH = "saved_models/MLP/"
os.makedirs(PATH, exist_ok=True)

results = []

for fname in os.listdir(DATA_DIR):
    if not fname.endswith('.csv'):
        continue
    print(f"\n--- Entrenando MLP con {fname} ---")
    df = pd.read_csv(os.path.join(DATA_DIR, fname))

    for pool in POOLS:
        for norm in NORMS:
            tr_ds, te_ds = prepare_mlp_data(df, pooling=pool, norm=norm, test_size=0.2)
            tr_loader = DataLoader(tr_ds, batch_size=BATCH, shuffle=True)
            te_loader = DataLoader(te_ds, batch_size=BATCH)

            model = MLPClassifier(tr_ds[0][0].shape[0]).to(DEVICE) 
            
            # Si LOAD_MODEL es True, intenta cargar un modelo preentrenado
            if LOAD_MODEL == True:
                model_path = os.path.join(PATH, f"mlp_{pool}_{norm}_{fname.replace('.csv', '')}.pth")
                if os.path.exists(model_path):
                    print(f"Cargando modelo preentrenado desde {model_path}")
                    model = load_model(MLPClassifier, model_path, input_dim=tr_ds[0][0].shape[0])
            
            history = run_training(model, tr_loader, te_loader, epochs=EPOCHS, lr=1e-3)

            results.append({
                'extractor':     fname,
                'model':         'MLP',
                'epochs':        EPOCHS,
                'batch_size':    BATCH,
                'input_dim':     tr_ds[0][0].shape[0],
                #'learning_rate': lr,
                'pooling':       pool,
                'normalization': norm,
                'train_loss':   round(history['train_loss'][-1], 5),
                'accuracy':      round(history['test_acc'][-1], 5),
                'f1_macro':      round(history['test_f1'][-1], 5)
            })
            
            # Guardar modelo
            #model_path = os.path.join(PATH, f"mlp_{pool}_{norm}_{fname.replace('.csv', '')}.pth")
            #save_model(model, model_path)
            
            

# Guardar resultados finales
df_results = pd.DataFrame(results)
results_path = "results/MLP/mlp_results.csv"

# Si el archivo existe, añadir los nuevos resultados al final
if os.path.exists(results_path):
    df_existing = pd.read_csv(results_path)
    
    with open(results_path, 'a', newline='') as f:
        f.write("\n") # Añadir una fila vacía para separar los resultados

    df_results.to_csv(results_path, mode='a', header=False, index=False)

    print(f"\n Resultados MLP añadidos a {results_path}")
else:
    # Si no existe, crear nuevo archivo
    df_results.to_csv(results_path, index=False)
    print(f"\n Nuevo archivo de resultados LSTM creado en {results_path}")

print("\n Resultados guardados en mlp_results.csv")



--- Entrenando MLP con baseline_CASIAB.csv ---
Ep01 | loss 1.0734 | acc 0.4828 | f1_macro 0.2171
Ep02 | loss 1.0278 | acc 0.4828 | f1_macro 0.2171
Ep03 | loss 0.9991 | acc 0.4828 | f1_macro 0.2171
Ep04 | loss 0.9748 | acc 0.4828 | f1_macro 0.2171
Ep05 | loss 0.9537 | acc 0.4943 | f1_macro 0.2390
Ep06 | loss 0.9306 | acc 0.4828 | f1_macro 0.2866
Ep07 | loss 0.8989 | acc 0.5057 | f1_macro 0.3240
Ep08 | loss 0.8658 | acc 0.5632 | f1_macro 0.3956
Ep09 | loss 0.8259 | acc 0.5747 | f1_macro 0.4111
Ep10 | loss 0.7850 | acc 0.5747 | f1_macro 0.4111
Ep11 | loss 0.7452 | acc 0.5517 | f1_macro 0.3942
Ep12 | loss 0.7070 | acc 0.5402 | f1_macro 0.3865
Ep13 | loss 0.6654 | acc 0.5057 | f1_macro 0.3645
Ep14 | loss 0.6215 | acc 0.5287 | f1_macro 0.4163
Ep15 | loss 0.5773 | acc 0.5172 | f1_macro 0.4060
Ep16 | loss 0.5355 | acc 0.5287 | f1_macro 0.4138
Ep17 | loss 0.4841 | acc 0.4943 | f1_macro 0.3912
Ep18 | loss 0.4405 | acc 0.5172 | f1_macro 0.4303
Ep19 | loss 0.3955 | acc 0.5172 | f1_macro 0.4303
Ep

C:\Users\Acer\AppData\Local\Temp\ipykernel_18352\3620464616.py:21: DtypeWarning: Columns (129) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(DATA_DIR, fname))


Ep01 | loss 1.1214 | acc 0.1818 | f1_macro 0.1558
Ep02 | loss 1.0999 | acc 0.2727 | f1_macro 0.1667
Ep03 | loss 1.0808 | acc 0.4545 | f1_macro 0.2083
Ep04 | loss 1.0635 | acc 0.5455 | f1_macro 0.2353
Ep05 | loss 1.0464 | acc 0.5455 | f1_macro 0.2353
Ep06 | loss 1.0292 | acc 0.5455 | f1_macro 0.2353
Ep07 | loss 1.0101 | acc 0.5455 | f1_macro 0.2353
Ep08 | loss 0.9927 | acc 0.5455 | f1_macro 0.2353
Ep09 | loss 0.9739 | acc 0.5455 | f1_macro 0.2353
Ep10 | loss 0.9539 | acc 0.5455 | f1_macro 0.2353
Ep11 | loss 0.9338 | acc 0.5455 | f1_macro 0.2353
Ep12 | loss 0.9109 | acc 0.5455 | f1_macro 0.2353
Ep13 | loss 0.8909 | acc 0.5455 | f1_macro 0.2353
Ep14 | loss 0.8683 | acc 0.5455 | f1_macro 0.2353
Ep15 | loss 0.8463 | acc 0.5455 | f1_macro 0.2353
Ep16 | loss 0.8234 | acc 0.5455 | f1_macro 0.2353
Ep17 | loss 0.8009 | acc 0.5455 | f1_macro 0.2353
Ep18 | loss 0.7778 | acc 0.5455 | f1_macro 0.2353
Ep19 | loss 0.7537 | acc 0.5455 | f1_macro 0.2353
Ep20 | loss 0.7292 | acc 0.5455 | f1_macro 0.2353


## Matriz de Confusión y Curva ROC

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

def evaluate_confusion_matrix(model, dataloader, device, class_names):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for batch in dataloader:
            # Adaptar según tu DataLoader (2 ó 3 elementos)
            if len(batch) == 3:
                xb, lengths, yb = batch
                xb, lengths = xb.to(device), lengths.to(device)
                logits = model(xb, lengths)
            else:
                xb, yb = batch
                xb = xb.to(device)
                logits = model(xb)

            preds = logits.argmax(dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(yb.numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    cm = confusion_matrix(y_true, y_pred)

    # Dibujar
    plt.figure(figsize=(5,5))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Matriz de Confusión")
    plt.colorbar()
    ticks = np.arange(len(class_names))
    plt.xticks(ticks, class_names, rotation=45)
    plt.yticks(ticks, class_names)
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     ha="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.ylabel("Etiqueta real")
    plt.xlabel("Etiqueta predicha")
    plt.tight_layout()
    plt.show()
    return cm



def evaluate_multiclass_roc(model, dataloader, device, class_names):
    model.eval()
    y_true, y_score = [], []

    with torch.no_grad():
        for batch in dataloader:
            if len(batch) == 3:
                xb, lengths, yb = batch
                xb, lengths = xb.to(device), lengths.to(device)
                logits = model(xb, lengths)
            else:
                xb, yb = batch
                xb = xb.to(device)
                logits = model(xb)

            probs = F.softmax(logits, dim=1).cpu().numpy()
            y_score.append(probs)
            y_true.append(yb.numpy())

    y_true = np.concatenate(y_true)
    y_score = np.concatenate(y_score)
    n_classes = y_score.shape[1]
    y_true_bin = label_binarize(y_true, classes=list(range(n_classes)))

    plt.figure(figsize=(6,5))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_score[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC={roc_auc:.2f})")

    plt.plot([0,1], [0,1], 'k--', label="Azar")
    plt.xlabel("Tasa Falsos Positivos")
    plt.ylabel("Tasa Verdaderos Positivos")
    plt.title("ROC Multi-clase (One-vs-Rest)")
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.show()


In [ ]:
# Asumiendo que ya tienes un modelo entrenado y un DataLoader de test
class_names = ['derecha', 'centro', 'izquierda']
evaluate_multiclass_roc(model, te_loader, DEVICE, class_names)
cm = evaluate_confusion_matrix(model, te_loader, DEVICE, class_names)

# Numero de aciertos y errores
print("\n=== Resultados de la matriz de confusión para test ===")
print("Aciertos (diagonal):", np.diag(cm).sum())
print("Errores (fuera de la diagonal):", cm.sum() - np.diag(cm).sum())


-------------------------------------------------------------------------------
## Modelo LSTM y BiLSTM

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(
        self,
        input_dim,      # 256 features del CSV
        hidden_dim,     # Dimensión oculta de la LSTM 
        num_layers,     # Número de capas LSTM 
        bidirectional,  # Si usar BiLSTM
        dropout        # Dropout entre capas y antes de clasificación
    ):
        super().__init__()

        # 1. Capas LSTM apiladas con dropout entre ellas
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=dropout if num_layers > 1 else 0.0  # Dropout solo si hay múltiples capas
        )
        
        # 2. Dimensiones de salida
        self.lstm_out_dim = hidden_dim * (2 if bidirectional else 1)  # Dimensión de salida LSTM
        self.hidden_dim = self.lstm_out_dim // 2 if bidirectional else self.lstm_out_dim

        # 3. Capa densa final
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.lstm_out_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, 3)  # 3 clases: derecha, centro, izquierda
        )

    def forward(self, x, lengths):
        # 1. Empaquetar secuencias variables
        packed = pack_padded_sequence(x, lengths.cpu(), 
                                    batch_first=True, 
                                    enforce_sorted=False)

        # 2. Procesar con LSTM
        # Shape de h_n: (num_layers * 2, batch_size, hidden_dim) el caso de BiLSTM
        # el num_layers * 2 es porque hay una capa forward y otra backward de ahí el 2
        _, (h_n, _) = self.lstm(packed)
        
        # 3. Obtener estado final (último estado de la última capa)
        if self.lstm.bidirectional:
            h_forward = h_n[-2]  # última capa forward
            h_backward = h_n[-1]  # última capa backward
            h_final = torch.cat([h_forward, h_backward], dim=1)
        else:
            h_final = h_n[-1]

        # 4. Pasar por la capa densa final
        return self.fc(h_final)


## Preparación de datos, split y bucle de entrenamiento: LSTM

**Normalización**: antes de la partición train/test se aplican dos esquemas alternativos de escalado MinMax (entrenando un MinMaxScaler sobre todos los frames y transformando cada secuencia) o L2-norm (normalizando cada frame por su norma L2) para unificar la escala de las características. Función: **`prepare_seq_data`**

**DataLoader con función `collate_sequences`**: cada lote de secuencias de longitud variable se rellena (pad) al tamaño de la más larga y se devuelve un tensor de longitudes, lo que permite a la LSTM procesar correctamente secuencias de distinta longitud en un mismo batch.

**Optimización con Adam, weight decay y grad-clip**: se entrena con Adam, añadiendo regularización L2 (weight_decay) y aplicando clip_grad_norm_ tras el backward para limitar la magnitud de los gradientes y prevenir explosiones que se manifiestan como picos abruptos en la train_loss. Función: **`run_training_lstm`**

**Early stopping**: tras 300 épocas sin una reducción significativa de la train_loss (más allá de un umbral min_delta), el entrenamiento se detiene para evitar sobreajuste y ahorro de recursos computacionales. `''`

**Seguimiento del mejor F1 con “warm-up”**: el cómputo de best_f1_score comienza únicamente a partir de la época 100. Esto impide registrar picos de F1 obtenidos por azar en fases iniciales, cuando el train_loss sigue siendo alto y el modelo aún no ha aprendido de forma estable. Establecer este mínimo de 100 épocas asegura que las mejoras de F1 reflejen un aprendizaje consolidado y verdaderamente generalizable. `''`

In [ ]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

def collate_sequences(batch):
    """
    Recibe una lista de (tensor_seq, label).
    Devuelve:
      - padded: tensor (B, T_max, D)
      - lengths: tensor (B,)
      - labels: tensor (B,)
    """
    seqs, labels = zip(*batch)
    lengths = torch.tensor([s.size(0) for s in seqs], dtype=torch.long)
    padded = pad_sequence(seqs, batch_first=True)
    labels = torch.tensor(labels, dtype=torch.long)
    return padded, lengths, labels

def prepare_seq_data(df, norm, test_size):
    """
    df: DataFrame con columnas feat_0…feat_D-1, video_ID y shoot_zone
    norm: 'minmax' o 'l2'
    Devuelve dos listas de muestras (tensor_seq, label) para train y test.
    """
    # 1) Extraer todas las secuencias y etiquetas
    feat_cols = [c for c in df.columns if c.startswith('feat_')]
    seqs, labs = [], []
    for vid, grp in df.groupby('video_ID'):
        arr = grp[feat_cols].values.astype(np.float32)  # (T, D)
        seqs.append(arr)
        labs.append(int(grp['shoot_zone'].iloc[0]))

    # 2) Normalizar
    if norm == 'minmax':
        all_frames = np.vstack(seqs)
        scaler = MinMaxScaler().fit(all_frames)
        seqs = [scaler.transform(s) for s in seqs]
    elif norm == 'L2':
        seqs = [normalize(s, norm='l2', axis=1) for s in seqs]

    # 3) Split estratificado
    idx = list(range(len(seqs)))
    idx_tr, idx_te = train_test_split(idx, test_size=test_size, stratify=labs, random_state=16)

    # 4) Convertir a listas de tuplas (tensor_seq, label)
    train_list = [(torch.from_numpy(seqs[i]), labs[i]) for i in idx_tr]
    test_list  = [(torch.from_numpy(seqs[i]), labs[i]) for i in idx_te]


    return train_list, test_list


def run_training_lstm(model, train_loader, test_loader, epochs, lr, wd, min_epochs):
    """
    Función de entrenamiento específica para el modelo LSTM
    Args:
        model: Instancia de LSTMClassifier
        train_loader: DataLoader con datos de entrenamiento (incluye lengths)
        test_loader: DataLoader con datos de test (incluye lengths)
        epochs: Número de épocas de entrenamiento
        lr: Learning rate para el optimizador
    Returns:
        history: Diccionario con métricas de entrenamiento
    """
    device = next(model.parameters()).device
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.CrossEntropyLoss()
    history = {'train_loss': [],'test_loss':[], 'test_acc': [], 'test_f1': []}

    patience = 50  # Paciencia para early stopping
    min_delta = 0.0001  # Mínima mejora para considerar que hay progreso
    best_loss = float('inf')  # Mejor pérdida inicial
    epochs_no_improve = 0  # Contador de épocas sin mejora

    best_f1_score = 0.0
    best_epoch = 0


    for ep in range(1, epochs + 1):
        # --- TRAIN ---
        model.train()
        total_loss = 0
        for xb, lengths, yb in train_loader:
            # Mover datos a GPU/CPU
            xb = xb.to(device)
            lengths = lengths.to(device)
            yb = yb.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            out = model(xb, lengths)
            loss = loss_fn(out, yb)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Clipping de gradientes para evitar explosiones
            optimizer.step()
            
            total_loss += loss.item() * yb.size(0)
        
        avg_loss = total_loss / len(train_loader.dataset)
        history['train_loss'].append(avg_loss)

        # --- TEST ---
        model.eval()
        all_preds, all_labels = [], []
        #test_sum = 0
        
        with torch.no_grad():
            for xb, lengths, yb in test_loader:
                xb = xb.to(device)
                lengths = lengths.to(device)
                
                # Predicción
                outputs = model(xb, lengths)
                preds = outputs.argmax(dim=1).cpu().numpy()
                
                #test_sum += loss_fn(outputs, yb).item() * yb.size(0)
                
                all_preds.append(preds)
                all_labels.append(yb.numpy())

        # Calcular métricas
        #avg_test_loss = test_sum / len(test_loader.dataset)
        #history['test_loss'].append(avg_test_loss)

        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)
        
        acc = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='macro')
        
        history['test_acc'].append(acc)
        history['test_f1'].append(f1)

        # Imprimir progreso
        print(f"Época {ep:02d}/{epochs} | "
              f"train_loss {avg_loss:.4f} | "
              f"accuracy {acc:.4f} | "
              f"f1_macro {f1:.4f}")
        
        # trackear best F1 solo tras un mínimo de épocas
        if ep > (min_epochs / 2) and f1 > best_f1_score + 1e-5:
            best_f1_score = f1
            best_epoch = ep

        # — Early stopping basado en train_loss — o se puede hacer para test_loss
        if ep > min_epochs:
            if avg_loss < best_loss - min_delta:
                best_loss = avg_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
            
            if avg_loss < 0.001:  # Si la pérdida es muy baja, detener
                print(f"\nEarly stopping tras {ep} épocas con train_loss={avg_loss:.4f}.")
                break

            if epochs_no_improve >= patience and avg_loss < 0.01: 
                print(f"\nEarly stopping tras {epochs_no_improve} épocas "
                      f"sin mejorar el train_loss (delta<{min_delta}).")
                break

    return history, ep, best_f1_score, best_epoch

# REVISAR: en vez de devolver ep y best_f1_score, devolver un diccionario con todo dentro de history


## Optimización de hiperparámetros

Se emplea StratifiedKFold sobre los IDs de vídeo (video_ID), no sobre los fotogramas individuales, para crear 5 particiones que mantienen la proporción de clases en cada fold. Para cada partición, se extraen dos listas de vídeos: una de entrenamiento y otra de validación, y se filtra el DataFrame original para incluir únicamente las secuencias completas de esos vídeos. Cada secuencia se normaliza (MinMax o L2) y se empaqueta en un DataLoader con collate_sequences, garantizando que ningún vídeo se mezcle entre train/val y preservando su continuidad temporal. Así se evita la fuga de información entre folds y se evalúa el modelo sobre vídeos inéditos en cada ronda.

In [ ]:
import os
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, normalize
import torch
from torch.utils.data import DataLoader

# Configuración global
DATA_DIR = "Gait_Embeddings_good"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def objective(trial, df):
    """Función objetivo para Optuna"""
    config = {
        'hidden_dim': trial.suggest_categorical("hidden_dim", [64, 128, 256, 512]),
        'num_layers': trial.suggest_int("num_layers", 1, 3),
        'bidirectional': trial.suggest_categorical("bidirectional", [False, True]),
        'dropout': trial.suggest_categorical("dropout", [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]),
        'lr': trial.suggest_categorical("lr", [1e-5, 1e-4, 1e-3]),
        'batch_size': trial.suggest_categorical("batch_size", [32, 64, 128]),
        'norm': trial.suggest_categorical("norm", ["minmax", "L2"])
    }

    print(f"\n{'='*50}")
    print(f"Trial {trial.number}")
    print(f"Configuración: {config}")
    print(f"{'='*50}\n")

    # Preparar datos para K-Fold
    video_ids = list(df.groupby('video_ID').groups.keys())
    labels = [int(df[df['video_ID']==vid]['shoot_zone'].iloc[0]) for vid in video_ids]

    # K-Fold Cross Validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=16)
    fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(video_ids, labels)):
        print(f"\nFold {fold+1}/5")
        
        # Separar datos de train/val
        train_ids = [video_ids[i] for i in train_idx]
        val_ids = [video_ids[i] for i in val_idx]
        
        df_train = df[df['video_ID'].isin(train_ids)]
        df_val = df[df['video_ID'].isin(val_ids)]
        
        # Preparar datos
        feat_cols = [c for c in df_train.columns if c.startswith('feat_')]
        
        # Train data
        train_seqs = []
        train_labs = []
        for vid, grp in df_train.groupby('video_ID'):
            arr = grp[feat_cols].values.astype(np.float32)
            if config['norm'] == 'minmax':
                arr = MinMaxScaler().fit_transform(arr)
            elif config['norm'] == 'L2':  
                arr = normalize(arr, norm='l2', axis=1)
            train_seqs.append(torch.from_numpy(arr))
            train_labs.append(int(grp['shoot_zone'].iloc[0]))
        
        # Val data
        val_seqs = []
        val_labs = []
        for vid, grp in df_val.groupby('video_ID'):
            arr = grp[feat_cols].values.astype(np.float32)
            if config['norm'] == 'minmax':
                arr = MinMaxScaler().fit_transform(arr)
            else:  # L2
                arr = normalize(arr, norm='l2', axis=1)
            val_seqs.append(torch.from_numpy(arr))
            val_labs.append(int(grp['shoot_zone'].iloc[0]))

        # Crear datasets y dataloaders
        train_data = list(zip(train_seqs, train_labs))
        val_data = list(zip(val_seqs, val_labs))
        
        train_loader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_sequences)
        val_loader = DataLoader(val_data, batch_size=config['batch_size'], shuffle=False, collate_fn=collate_sequences)

        # Crear y entrenar modelo
        model = LSTMClassifier(
            input_dim=df.filter(like='feat_').shape[1],
            hidden_dim=config['hidden_dim'],
            num_layers=config['num_layers'],
            bidirectional=config['bidirectional'],
            dropout=config['dropout']
        ).to(DEVICE)

        # Entrenar
        history, _, _, _ = run_training_lstm(
            model=model,
            train_loader=train_loader, 
            test_loader=val_loader,
            epochs=200,  
            lr=config['lr'],
            wd=0, # No usamos weight decay aquí
            min_epochs=100 # para early stopping

        )
        
        # Guardar mejor F1 score del fold
        fold_scores.append(max(history['test_f1']))

    mean_f1 = np.mean(fold_scores)
    print(f"\nConfiguración: {config}")
    print(f"F1-score medio: {mean_f1:.4f}")
    
    return mean_f1



def optimize_embeddings():
    """Ejecuta optimización para cada embedding base"""
    
    os.makedirs("results", exist_ok=True)
    results = {}
    
    for fname in os.listdir(DATA_DIR):
        print(f"\n{'='*50}")
        print(f"Optimizando {fname}")
        print(f"{'='*50}")
        
        # Cargar datos
        df = pd.read_csv(os.path.join(DATA_DIR, fname))
        
        # Crear y ejecutar estudio
        study = optuna.create_study(direction="maximize")
        study.optimize(
            lambda trial: objective(trial, df),
            n_trials=20,  # 30 trials por embedding
            n_jobs=10,     # Paralelización
            timeout=3600  # Timeout de 1 hora por embedding
        )
        
        # Guardar resultados
        results[fname] = {
            'best_params': study.best_params,
            'best_f1': study.best_value
        }
        
        # Guardar trials en CSV
        study_df = study.trials_dataframe()
        study_df.to_csv(f"results/LSTM_hyperparams_{fname}.csv", index=False)
        
        print(f"\nMejores parámetros para {fname}:")
        print(f"F1-score: {study.best_value:.4f}")
        print("Configuración:", study.best_params)
        
    # Guardar resumen final
    final_results = []
    for embedding, res in results.items():
        row = {
            'embedding': embedding,
            'best_f1': res['best_f1'],
            **res['best_params']
        }
        final_results.append(row)
    
    df_final = pd.DataFrame(final_results)
    df_final.to_csv("results/best_parameters/best_params_LSTM.csv", index=False)
    print("\nResumen final guardado en results/best/parameters/best_params_LSTM.csv")
    
    return results

In [ ]:
# Optuna
results = optimize_embeddings()

## Entrenamiento 
Configuración de hiperparámetros a partir de los resultados obtenidos mediante Optuna.
- Mencionar que las métricas que se añaden a los resultados obtenidos son las siguientes:
- f1-score...

Usando clip_grad_norm tampoco se obtenia apenas diferencia en los resultados.

In [ ]:

# Configuración 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = "Gait_Embeddings_good"
EPOCHS = 1500
BATCH = 32
LR = 1e-3
WD = 0.0
NORM = 'L2'
F1_THRESHOLD = 0.5  # Umbral mínimo de F1 para considerar un modelo válido
LOAD_MODEL = False  # Si True, carga un modelo preentrenado en lugar de entrenar uno nuevo


# Hiperparámetros de LSTM
num_layers    = 1
hidden_dim    = 512
bidirectional = True
dropout       = 0.0


results_lstm = []


for fname in os.listdir(DATA_DIR):
    if not fname.endswith('.csv'):
        continue
        
    print(f"\n--- Entrenando LSTM con {fname} ---")
    df = pd.read_csv(os.path.join(DATA_DIR, fname))
    
    # Preparar datos
    train_list, test_list = prepare_seq_data(df, norm=NORM, test_size=0.2)
    tr_loader = DataLoader(train_list, batch_size=BATCH, shuffle=True, collate_fn=collate_sequences)
    te_loader = DataLoader(test_list, batch_size=BATCH, shuffle=False, collate_fn=collate_sequences)

    # Crear y entrenar modelo
    model = LSTMClassifier(
        input_dim=df.filter(like='feat_').shape[1],
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        bidirectional=bidirectional,
        dropout=dropout
    ).to(DEVICE)
    
    # REVISAR EL MEOTDO DE CARGA DEL MODELO NO VA BIEN
    # Si LOAD_MODEL es True, intenta cargar un modelo preentrenado
    if LOAD_MODEL:
        model_path = os.path.join("saved_models/LSTM", f"lstm_{NORM}_hidden{hidden_dim}_layers{num_layers}_batch{BATCH}_LR{LR}_dropout{dropout}_BiLSTM_{bidirectional}_{fname.replace('.csv', '')}.pth")
        if os.path.exists(model_path):
            print(f"Cargando modelo preentrenado desde {model_path}")
            model = load_model(LSTMClassifier, model_path, input_dim=df.filter(like='feat_').shape[1])

    history, ep, best_f1, best_epoch = run_training_lstm(model, tr_loader, te_loader, epochs=EPOCHS, lr=LR, wd=WD, min_epochs=100) 


    # Guardar resultados
    results_lstm.append({
        'extractor': fname,
        'model': 'LSTM',
        'epochs': int(ep),
        'batch_size': int(BATCH),
        'normalization': NORM,
        'num_layers': int(num_layers),
        'hidden_dim': int(hidden_dim),
        'bidirectional': bidirectional,
        'dropout': dropout,
        'lr': LR,
        'train_loss': round(history['train_loss'][-1], 5),
        'accuracy': round(history['test_acc'][-1], 5),
        'f1_macro': round(history['test_f1'][-1], 5),
        
        'best_f1_score': round(best_f1, 5),
        'best_epoch': best_epoch
    })

    if best_f1 < F1_THRESHOLD:
        print(f"\nModelo con F1 {best_f1:.4f} por debajo del umbral {F1_THRESHOLD}. No se guardará.")
        continue

    # Guardar modelo
    model_path = os.path.join("saved_models/LSTM", f"lstm_{NORM}_hidden{hidden_dim}_layers{num_layers}_batch{BATCH}_LR{LR}_dropout{dropout}_BiLSTM_{bidirectional}_{fname.replace('.csv', '')}.pth")
    save_model(model, model_path)


df_results = pd.DataFrame(results_lstm)
results_path = "results/LSTM/lstm_results.csv"

# Si el archivo existe, añadir los nuevos resultados al final
if os.path.exists(results_path):
    dtype_dict = {
        'epochs': 'Int64',
        'batch_size': 'Int64',
        'num_layers': 'Int64',
        'hidden_dim': 'Int64',
        'best_epoch': 'Int64'
    }
    df_existing = pd.read_csv(results_path, dtype=dtype_dict)
    
    with open(results_path, 'a', newline='') as f:
        f.write("\n") # Añadir una fila vacía para separar los resultados

    df_results.to_csv(results_path, mode='a', header=False, index=False)

    print(f"\n Resultados LSTM añadidos a {results_path}")
else:
    # Si no existe, crear nuevo archivo
    df_results.to_csv(results_path, index=False)
    print(f"\n Nuevo archivo de resultados LSTM creado en {results_path}")

# Mostrar todos los resultados
print("\nResumen de todos los resultados:")
print(pd.read_csv(results_path))


## Matriz de confusión

In [ ]:
# Asumiendo que ya tienes un modelo entrenado y un DataLoader de test
class_names = ['derecha', 'centro', 'izquierda']
evaluate_multiclass_roc(model, te_loader, DEVICE, class_names)
cm = evaluate_confusion_matrix(model, te_loader, DEVICE, class_names)

# Numero de aciertos y errores
print("\n=== Resultados de la matriz de confusión para test ===")
print("Aciertos (diagonal):", np.diag(cm).sum())
print("Errores (fuera de la diagonal):", cm.sum() - np.diag(cm).sum())


-------------------------------------------------------------------------------
## Modelo TCN - Temporal Convolutional Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm

class Chomp1d(nn.Module):
    """Elimina el exceso de padding al final de la secuencia."""
    def __init__(self, chomp_size):
        super().__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        # x tiene forma (batch, channels, seq_len + 2*(kernel_size-1)*dilation)
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    """Bloque residual de la TCN."""
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding, dropout):
        super().__init__()
        self.conv1 = weight_norm(nn.Conv1d(in_channels, out_channels,
                                           kernel_size,
                                           stride=stride,
                                           padding=padding,
                                           dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(out_channels, out_channels,
                                           kernel_size,
                                           stride=stride,
                                           padding=padding,
                                           dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(dropout)

        # Si cambia el número de canales, ajustamos la rama de shortcut
        self.downsample = (nn.Conv1d(in_channels, out_channels, 1)
                           if in_channels != out_channels else None)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.chomp1(out)
        out = self.relu1(out)
        out = self.drop1(out)

        out = self.conv2(out)
        out = self.chomp2(out)
        out = self.relu2(out)
        out = self.drop2(out)

        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    """Stack de bloques temporales con dilataciones crecientes."""
    def __init__(self, num_inputs, num_channels, kernel_size=3, dropout=0.2):
        """
        num_inputs: dimensión de entrada (features)
        num_channels: lista con número de filtros por capa, p.ej. [128, 128, 128]
        """
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            in_ch = num_inputs if i == 0 else num_channels[i-1]
            out_ch = num_channels[i]
            dilation = 2 ** i
            padding = (kernel_size - 1) * dilation
            layers += [TemporalBlock(in_ch, out_ch, kernel_size, stride=1,
                                     dilation=dilation, padding=padding,
                                     dropout=dropout)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        # x: (batch, seq_len, features) → lo ponemos (batch, features, seq_len)
        x = x.transpose(1, 2)
        y = self.network(x)
        # devolvemos (batch, out_ch, seq_len)
        return y

class TCNClassifier(nn.Module):
    """TCN seguido de pooling global y capa de salida."""
    def __init__(self, input_dim, num_classes, num_channels, kernel_size=3, dropout=0.2):
        """
        input_dim: dimensión de cada vector temporal (features)
        num_classes: número de clases de salida
        num_channels: lista de canales en cada bloque, p.ej. [128, 128, 128]
        """
        super().__init__()
        self.tcn = TemporalConvNet(input_dim, num_channels,
                                   kernel_size=kernel_size,
                                   dropout=dropout)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(num_channels[-1], num_classes)

    def forward(self, x, lengths=None):
        """
        x: tensor (batch, seq_len, input_dim)
        lengths: opcional, no usado aquí
        """
        y = self.tcn(x)                   # (batch, C, seq_len)
        y = self.global_pool(y).squeeze(-1)  # (batch, C)
        out = self.fc(y)                  # (batch, num_classes)
        return out


In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm

class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout=0.2):
        super(TemporalBlock, self).__init__()
        # Capa convolucional 1D causal con padding = (kernel_size-1)*dilación (para asegurar causalidad)
        self.conv1 = weight_norm(nn.Conv1d(in_channels, out_channels, kernel_size, 
                                           padding=(kernel_size-1)*dilation, dilation=dilation))
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        # Segunda capa convolucional en el bloque (otro nivel de no linealidad)
        self.conv2 = weight_norm(nn.Conv1d(out_channels, out_channels, kernel_size, 
                                           padding=(kernel_size-1)*dilation, dilation=dilation))
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        # Combinamos las capas en un bloque secuencial
        self.net = nn.Sequential(self.conv1, self.relu1, self.dropout1,
                                 self.conv2, self.relu2, self.dropout2)
        # Conexión residual: si cambia el número de canales, ajustamos con conv 1x1
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()
        
    def forward(self, x):
        """
        Propagación hacia adelante del bloque temporal.
        Entrada x de tamaño (batch, in_channels, seq_len).
        """
        out = self.net(x)
        # Recorte para causalidad: después de conv con padding, las salidas más allá de la longitud original se descartan
        if self.conv1.padding[0] > 0:
            out = out[:, :, :-self.conv1.padding[0]]  # elimina "efecto futuro"
        # Suma residual (skip connection)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)  # activación ReLU tras sumar residual

class TemporalConvNet(nn.Module):
    def __init__(self, input_channels, channel_sizes, kernel_size=3, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(channel_sizes)
        # Construye múltiples bloques temporales con dilaciones crecientes (potencias de 2)
        for i in range(num_levels):
            in_ch = input_channels if i == 0 else channel_sizes[i-1]
            out_ch = channel_sizes[i]
            dilation = 2 ** i  # dilatación creciente por nivel (1, 2, 4, ...)
            layers.append(TemporalBlock(in_ch, out_ch, kernel_size, dilation, dropout))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        # Pasa la entrada a través de la pila de bloques temporales
        return self.network(x)

class TCNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, channel_sizes=[32, 32, 32], kernel_size=3, dropout=0.2):
        """
        input_dim: número de características de entrada por tiempo (ej: dimensión del embedding por frame).
        num_classes: número de clases de salida (ej: posibles zonas de tiro en el penalti).
        channel_sizes: lista con el número de filtros en cada capa TCN.
        """
        super(TCNClassifier, self).__init__()
        self.tcn = TemporalConvNet(input_dim, channel_sizes, kernel_size, dropout)
        # Capa lineal final para producir la predicción de clase a partir de la representación temporal
        final_out_channels = channel_sizes[-1]
        self.fc = nn.Linear(final_out_channels, num_classes)
    
    def forward(self, x):
        """
        x tiene tamaño (batch, input_dim, seq_len).
        """
        features = self.tcn(x)         # salida TCN: (batch, final_out_channels, seq_len)
        final_feature = features[:, :, -1]  # tomamos la característica del último paso temporal (último frame)
        out = self.fc(final_feature)   # predicción de la clase
        return out

# ==== Ejemplo de instanciación y uso ====
batch_size = 8
seq_len = 30       # por ejemplo, 30 frames de entrada
input_dim = 50      # 50 características por frame (ej: coordenadas, ángulos, etc.)
num_classes = 5     # 5 posibles zonas de lanzamiento (clases)

model = TCNClassifier(input_dim, num_classes, channel_sizes=[32, 32, 64], kernel_size=3, dropout=0.3)
# Datos de ejemplo aleatorios
example_input = torch.rand(batch_size, input_dim, seq_len)  # tensor de tamaño (8, 50, 30)
logits = model(example_input)  # salida del modelo (8, 5) sin activar (logits de clase)
print(logits.shape)  # debería ser [8, 5]


-------------------------------------------------------------------------------
## Modelo Transformer

In [ ]:

# 3) Transformer con atención temporal
class TransformerClassifier(nn.Module):
    def __init__(
        self,
        input_dim,
        model_dim=256,
        n_heads=4,
        num_layers=2,
        ff_dim=512,
        dropout=0.1
    ):
        """
        input_dim: dimensión D del embedding sin pooling (secuencia)
        model_dim: dimensión interna del transformer
        n_heads: número de cabezas de atención
        num_layers: número de capas Encoder
        ff_dim: dimensión del feed-forward
        dropout: dropout en capas Encoder
        """
        super().__init__()
        # Proyección de la dimensión de entrada al espacio model_dim
        self.token_proj = nn.Linear(input_dim, model_dim)
        # Capa TransformerEncoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead=n_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )
        # Clasificador al token de posición 0 (CLS) o media de salidas
        self.classifier = nn.Linear(model_dim, 3)

    def forward(self, x, lengths=None):
        """
        x: FloatTensor (B, T, D)
        lengths: LongTensor (B,) opcional para masking
        """
        # x → proyección
        x = self.token_proj(x)  # (B, T, model_dim)

        # Generar máscara de padding si lengths dado
        if lengths is not None:
            max_len = x.size(1)
            mask = torch.arange(max_len, device=lengths.device) \
                   .unsqueeze(0) >= lengths.unsqueeze(1)
        else:
            mask = None

        # TransformerEncoder
        out = self.transformer(x, src_key_padding_mask=mask)  # (B, T, model_dim)

        # Agregado temporal: tomamos token 0 como representativo
        cls_token = out[:, 0, :]  # (B, model_dim)
        return self.classifier(cls_token)



## Preparación de datos, split y bucle de entrenamiento: Transformer
Con atención atemporal


In [ ]:
# Chunk 2: Collate function y preparación de datos para Transformer

import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

def collate_sequences(batch):
    """
    Recibe una lista de tuplas (tensor_seq, label).
    Devuelve:
      - padded: FloatTensor (B, T_max, D)
      - lengths: LongTensor (B,)
      - labels: LongTensor (B,)
    """
    seqs, labels = zip(*batch)
    lengths = torch.tensor([s.size(0) for s in seqs], dtype=torch.long)
    padded  = pad_sequence(seqs, batch_first=True)
    labels  = torch.tensor(labels, dtype=torch.long)
    return padded, lengths, labels

def prepare_transformer_data(df, norm='minmax', test_size=0.2):
    """
    Construye listas de muestras para train y test:
      - norm: 'minmax' o 'l2'
    Cada muestra es (tensor_seq, label).
    """
    feat_cols = [c for c in df.columns if c.startswith('feat_')]
    seqs, labs = [], []
    for vid, grp in df.groupby('video_ID'):
        arr = grp[feat_cols].values.astype(np.float32)  # (T, D)
        seqs.append(arr)
        labs.append(int(grp['shoot_zone'].iloc[0]))
    # Normalización
    if norm == 'minmax':
        all_frames = np.vstack(seqs)  # (sum_T, D)
        scaler = MinMaxScaler().fit(all_frames)
        seqs = [scaler.transform(s) for s in seqs]
    else:  # 'l2'
        seqs = [s / np.linalg.norm(s, axis=1, keepdims=True) for s in seqs]
    # Split estratificado
    idx = list(range(len(seqs)))
    idx_tr, idx_te = train_test_split(idx,
                                      test_size=test_size,
                                      stratify=labs,
                                      random_state=42)
    # Convertir a lista de tuplas (tensor_seq, label)
    train_list = [(torch.from_numpy(seqs[i]), labs[i]) for i in idx_tr]
    test_list  = [(torch.from_numpy(seqs[i]), labs[i]) for i in idx_te]
    return train_list, test_list


def run_training_transformer(model, train_loader, test_loader, epochs=20, lr=1e-3):
    """
    Función de entrenamiento específica para el modelo Transformer
    Args:
        model: Instancia de TransformerClassifier
        train_loader: DataLoader con datos de entrenamiento 
        test_loader: DataLoader con datos de test
        epochs: Número de épocas de entrenamiento
        lr: Learning rate para el optimizador
    Returns:
        history: Diccionario con métricas de entrenamiento
    """
    device = next(model.parameters()).device
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    history = {'train_loss': [], 'test_acc': [], 'test_f1': []}

    for ep in range(1, epochs + 1):
        # --- Fase de entrenamiento ---
        model.train()
        total_loss = 0
        for xb, lengths, yb in train_loader:
            xb = xb.to(device)
            lengths = lengths.to(device)
            yb = yb.to(device)
            
            optimizer.zero_grad()
            out = model(xb, lengths)
            loss = loss_fn(out, yb)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * yb.size(0)
        
        avg_loss = total_loss / len(train_loader.dataset)
        history['train_loss'].append(avg_loss)

        # --- Fase de evaluación ---
        model.eval()
        all_preds, all_labels = [], []
        
        with torch.no_grad():
            for xb, lengths, yb in test_loader:
                xb = xb.to(device)
                lengths = lengths.to(device)
                outputs = model(xb, lengths)
                preds = outputs.argmax(dim=1).cpu().numpy()
                all_preds.append(preds)
                all_labels.append(yb.numpy())

        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)
        
        acc = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='macro')
        
        history['test_acc'].append(acc)
        history['test_f1'].append(f1)

        print(f"Época {ep:02d}/{epochs} | "
                f"loss {avg_loss:.4f} | "
                f"acc {acc:.4f} | "
                f"f1_macro {f1:.4f}")

    return history


## Entrenamiento

In [ ]:
import os
import pandas as pd
from torch.utils.data import DataLoader

# Configuración

DATA_DIR = "Gait_Embeddings_good"
NORMS = ['minmax', 'l2']
EPOCHS = 100  # Aumentado para mejor convergencia
BATCH_SIZE = 32  # Tamaño de batch para entrenamiento
LR = 1e-3
MODEL_DIM = 256  # Dimensión del transformer
N_HEADS = 8  # Número de cabezas de atención
N_LAYERS = 2  # Capas del encoder

results_transformer = []

for fname in os.listdir(DATA_DIR):
    if not fname.endswith('.csv'):
        continue
        
    print(f"\n--- Entrenando Transformer con {fname} ---")
    df = pd.read_csv(os.path.join(DATA_DIR, fname))
    
    for norm in NORMS:
        # Preparar datos
        train_list, test_list = prepare_transformer_data(df, norm=norm)
        tr_loader = DataLoader(train_list, 
                             batch_size=BATCH_SIZE, 
                             shuffle=True, 
                             collate_fn=collate_sequences)
        te_loader = DataLoader(test_list, 
                             batch_size=BATCH_SIZE,
                             collate_fn=collate_sequences)

        # Crear y entrenar modelo
        model = TransformerClassifier(
            input_dim=df.filter(like='feat_').shape[1],
            model_dim=MODEL_DIM,
            n_heads=N_HEADS,
            num_layers=N_LAYERS,
            ff_dim=MODEL_DIM * 4,  # Típicamente 4x model_dim
            dropout=0.1
        ).to(DEVICE)
        
        history = run_training_transformer(
            model, 
            tr_loader, 
            te_loader,
            epochs=EPOCHS, 
            lr=LR
        )

        # Guardar resultados
        results_transformer.append({
            'extractor': fname,
            'model': 'Transformer',
            'normalization': norm,
            'accuracy': history['test_acc'][-1],
            'f1_macro': history['test_f1'][-1]
        })

# Guardar resultados
df_results = pd.DataFrame(results_transformer)
df_results.to_csv('transformer_results.csv', index=False)
print("\n✅ Resultados Transformer guardados en transformer_results.csv")

## Métricas posibles a usar
Precisión global (Accuracy)

F₁‐score macro
Matriz de confusión

Precisión (Precision) por clase

Exhaustividad (Recall) por clase

Balanced accuracy (accuracy balanceada)

Matthew’s Correlation Coefficient (MCC)

Curva ROC y AUC multiclass (one-vs-rest)

Log-Loss (Cross-Entropy Loss)

Brier Score

Cohen’s Kappa

Top-k accuracy (por ejemplo Top-2)

Time-to-decision (número medio de frames o ms antes del golpeo en que la predicción es estable)

Área bajo la curva Accuracy vs. Earliness

Tiempo de inferencia por muestra (latencia)

Número de parámetros / FLOPS / uso de memoria